# How to use DEAP for Genetic Programming (GP)

By Claus Aranha (caranha), 2022-11-14

A reference for using the DEAP library

In [1]:
# Importing necessary modules

# Python operators: Add, Multiply, Subtract, etc.
import operator 
import random   

# Modules from the DEAP library for using Genetic Programming
from deap import base, gp, tools

# Module for generating Graphs with GraphViz (requires external installation)
import pygraphviz as pgv


## 1. Defining a GP Primitive set

The key choice when implementing GP is they **Primitive Set**. There are two types of primitives:

- Terminals: Inputs and Constants
- Operators: Modify the terminals

In DEAP, the primitive set is contained in a `gp.PrimitiveSet` object. 

- Creating Terminals: The object creator will create the necessary number of input terminals, and there are function for adding constant terminals. 
- Creating Operators: Any python function can be added as an operator, including user defined functions, using the `pset.addPrimitive` function. You must remember to set the **arity**, which is the number of parameters.




In [2]:
# Create the Primitive Set Object, with the number of inputs:
pset = gp.PrimitiveSet("main", 2, "IN") # The third parameter is the name we give to the input terminals

## Adding constants:
# Fixed value terminal
pset.addTerminal(1)            
# An Ephemeral Constant has a random value when it is created
pset.addEphemeralConstant("EPC", lambda: random.randint(-1,1)) 

## We can also change the name of the terminals if we want:
# pset.renameArguments(IN0="x") # Setting the input var name
# pset.renameArguments(IN1="y") # Setting the input var name

## Adding Operators:

# Operators from python functions:
pset.addPrimitive(max, 2)          # add the max function as a primitive with 2 inputs
pset.addPrimitive(operator.add, 2) # add the addition operator (2 inputs)
pset.addPrimitive(operator.mul, 2) # add the mult operator (2 inputs)
pset.addPrimitive(operator.neg, 1) # add the negation operator (1 input)

# Operators from hand-made functions:
def divideByTwo(n):
    return n / 2.

pset.addPrimitive(divideByTwo, 1)



## 2. Generating a Tree

There are two functions in DEAP for generating GP trees: 

- `gp.genFull(pset, min_, max_)`: Generates a full tree. Every terminal has the same depth (between min and max)
- `gp.genGrow(pset, min_, max_)`: Generates a tree. Every terminal has different depth (between min and max)

These two functions generate a list of DEAP objects (operators and terminals). You may want to convert these into the more complete `PrimitiveTree` object.

In [3]:
# Generating a Tree

expr_full = gp.genFull(pset, min_ = 2, max_ = 2)
expr_grow = gp.genGrow(pset, min_ = 2, max_ = 4)

tree_full = gp.PrimitiveTree(expr_full)
tree_grow = gp.PrimitiveTree(expr_grow)

print(tree_full)
print(tree_grow)

neg(add(0, IN0))
max(max(max(neg(-1), add(IN1, IN0)), divideByTwo(IN1)), divideByTwo(-1))


## 3. Visualizing a Tree

It is **Very Important** to visualize the results obtained by Genetic Programming. 

You can use the `gp.graph(tree)` function to generate the list of nodes, edges, and labels of a GP tree.

In [4]:
# Tree Visualization
nodes, edges, labels = gp.graph(tree_grow)

print(nodes)
print(edges)
print(labels)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
[(0, 1), (1, 2), (2, 3), (3, 4), (2, 5), (5, 6), (5, 7), (1, 8), (8, 9), (0, 10), (10, 11)]
{0: 'max', 1: 'max', 2: 'max', 3: 'neg', 4: -1, 5: 'add', 6: 'IN1', 7: 'IN0', 8: 'divideByTwo', 9: 'IN1', 10: 'divideByTwo', 11: -1}


### 3.1 GP Tree visualization with GraphViz

`GraphViz` provides a very easy way to generate images from trees.

You can, of course, use any program that you want to visualize a GP tree Graph.

In [5]:
# Graphviz

def drawtree(t, filename):
    nodes, edges, labels = gp.graph(t)
    g = pgv.AGraph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    g.layout(prog="dot")

    for i in nodes:
        n = g.get_node(i)
        n.attr["label"] = labels[i]

    g.draw(filename)

drawtree(tree_full, "tree_full.png")
drawtree(tree_grow, "tree_grow.png")


[Tree_full](tree_full.png)  
[Tree_grow](tree_grow.png)

## 4. Evaluating a Tree

We can create a function from a GP tree using the `gp.compile(tree, pset)` function.

In [8]:
# Compiling the Tree
tg_func = gp.compile(tree_grow, pset)

# Executing the Tree:
print("Function: {}".format(tree_grow))
print("Result for (1, 2): {}".format(tg_func(1, 2)))

Function: max(max(max(neg(-1), add(IN1, IN0)), IN0), divideByTwo(-1))
Result for (1, 2): 3


## 5. Crossover and Mutation

There are several crossover and mutation operators in deap. Let's see two of them.

- `gp.cxOnePoint(tree1, tree2)`: chooses the crossover point between tree 1 and tree 2 at random, and create two new trees.
- `gp.mutInsert(tree, pset)`: creates a new branch at a random point in the tree

In [21]:
# Crossover:

cx_tree1, cx_tree2 = gp.cxOnePoint(tree_grow, tree_full)

drawtree(cx_tree1, "crossover_tree1.png")
drawtree(cx_tree2, "crossover_tree2.png")

# Mutation

mut_tree, = gp.mutInsert(tree_grow, pset)

drawtree(mut_tree, "mutation_tree.png")

[Crossover 1](crossover_tree1.png)  
[Crossover 2](crossover_tree2.png)  
[Mutation](mutation_tree.png)

## 6. Putting it all together

- Create a list, `pop` of random initial trees.
- For loop:
  - Evaluate `pop`
  - Record the best individual so far
  - Report average and best fitness
  - Select some trees from pop based on fitness
  - Create `pop_new` by applying crossover and mutation on trees from `pop`
  - `pop = pop_new`
- Report the final best individual

You can create this loop by hand, or you can use functions from DEAP. 

We will see both options.

## Extra: Loading a tree from a string

After you finish your computation, you want to save the best program that you found. You can save it as a string, and load it later using `gp.PrimitiveTree.from_string(string, pset)`. Don't forget to create the same pset (or save it using pickle or something).

In [23]:
# We can create trees directly from strings.

tree_string = gp.PrimitiveTree.from_string("mul(IN0, IN1)", pset)
print(tree_string)

ts_func = gp.compile(tree_string, pset)
print(ts_func(3, 2))

drawtree(tree_string, "tree_string.png")

mul(IN0, IN1)
6


[String Tree](tree_string.png)